In [1]:
import numpy as np 
import pandas as pd 
import os
from pathlib import Path
import timm
import random
from typing import List
import glob

from fastai.basics           import *
from fastai.medical.imaging  import *

from dataclasses import dataclass, field, asdict

import logging
from transformers import Trainer, TrainingArguments

import albumentations as A

import sklearn
from sklearn.model_selection import GroupKFold

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)

/opt/conda/lib/python3.10/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


# config

In [2]:
DEBUG = False

@dataclass
class DatasetConfig:
    path: Path = Path('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/')
    label_columns = [
       'left_neural_foraminal_narrowing_l1_l2',
       'left_neural_foraminal_narrowing_l2_l3',
       'left_neural_foraminal_narrowing_l3_l4',
       'left_neural_foraminal_narrowing_l4_l5',
       'left_neural_foraminal_narrowing_l5_s1',
       'right_neural_foraminal_narrowing_l1_l2',
       'right_neural_foraminal_narrowing_l2_l3',
       'right_neural_foraminal_narrowing_l3_l4',
       'right_neural_foraminal_narrowing_l4_l5',
       'right_neural_foraminal_narrowing_l5_s1']
    label_map = {
        'Normal/Mild': 0,
        'Moderate': 1,
        'Severe': 2,
        -100: -100 # for missing values
    }
    # use_n_images: int = 18  # select images

@dataclass
class ModelConfig:
    name: str = 'efficientnet_b3.ra2_in1k' # resnet or efficientnet
    train_backbone: bool = False # fine-tuned?
    use_weights: bool = True  # use weighted loss?

@dataclass
class Config:
    dataset: DatasetConfig = field(default_factory=DatasetConfig)
    model: ModelConfig = field(default_factory=ModelConfig)
    seed: int = 42
    n_folds: int = 5
        
args = TrainingArguments(
    output_dir='/kaggle/working/',
    do_eval=True,
    report_to='none',
    remove_unused_columns=False,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
#     torch_compile=True,
    num_train_epochs=5,
    warmup_ratio=0.05,
    logging_strategy='steps',
    logging_steps=100,
   # save_strategy='epoch',
    save_strategy='steps',
    save_steps=100,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_safetensors=False,
#     fp16=True,
    metric_for_best_model='log_loss',
#    eval_strategy='epoch',
    eval_strategy='steps',          # 设置为 'steps'，每隔一定batch评估一次
    eval_steps=100,                  # 每10个 batch 评估一次
    dataloader_num_workers=0,
    label_names=['labels'],
#     optim="adafactor",
)

config = Config()

wandb_config = {
    'config': asdict(config),
    'training_args': asdict(args)
}

import datetime
NAME = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# build model ResNet

In [3]:
config.model.name = 'resnet50'
backbone = timm.create_model(config.model.name, pretrained=True)

if not config.model.train_backbone:
    for param in backbone.parameters():
        param.require_grad = False

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(backbone)
transforms = timm.data.create_transform(**data_config, is_training=False)

transforms.transforms[-1].mean = transforms.transforms[-1].mean.mean()
transforms.transforms[-1].std = transforms.transforms[-1].std.mean()

IMG_WIDTH, IMG_HEIGHT = transforms.transforms[1].size

class Model1_ResNet(nn.Module):
    def __init__(self, backbone):
        super(Model1_ResNet, self).__init__()
        self.name = "ResNet"
        self.backbone = backbone
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.head = nn.Linear(self.backbone.num_features, len(config.dataset.label_columns)*3)

    def forward(self, **kwargs):
        batch = dict(**kwargs)
        # x: (B, N_IMAGES, C, H, W)
        features_output = []
        for series_description in ['Sagittal T2/STIR', 'Sagittal T1', 'Axial T2']:
            if not series_description in batch:
                # not used, we just forward zeros through backbone
                features_output.append(None)
            else:
                images = batch[series_description]
                B, N_IMAGES, C, H, W = images.shape
                if config.model.train_backbone:
                    features = self.backbone.forward_features(images.view(B * N_IMAGES, C, H, W))
                else:
                    with torch.no_grad():
                        features = self.backbone.forward_features(images.view(B * N_IMAGES, C, H, W))
                features = self.pool(features).squeeze(-1).squeeze(-1)
                features = self.head(features).view(B, N_IMAGES, -1)
                # features: (B, N_IMAGES, label_columns * 3)
                features_output.append(features)
        for i in range(len(features_output)):
            if features_output[i] is None:
                features_output[i] = torch.zeros((B, N_IMAGES, self.backbone.num_features), device=images.device)
        features_output = torch.cat(features_output, 1)
        output = features_output.mean(1)
        output = output.view(-1, 10, 3)

        loss = None
        if 'labels' in batch:
            if config.model.use_weights:
                loss_fn = torch.nn.CrossEntropyLoss(torch.tensor([1., 2., 4.], device=output.device))
            else:
                loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(output.view(-1, 3), batch['labels'].view(-1, ))

        return loss, output

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

# build model efficient+lstm

@todo

In [4]:
config.model.name = 'efficientnet_b3.ra2_in1k'
backbone = timm.create_model(config.model.name, pretrained=True)

if not config.model.train_backbone:
    for param in backbone.parameters():
        param.require_grad = False

data_config = timm.data.resolve_model_data_config(backbone)
transforms = timm.data.create_transform(**data_config, is_training=False)

transforms.transforms[-1].mean = transforms.transforms[-1].mean.mean()
transforms.transforms[-1].std = transforms.transforms[-1].std.mean()

IMG_WIDTH, IMG_HEIGHT = transforms.transforms[1].size

class Model2_LSTM(nn.Module):
    def __init__(self, backbone):
        super(Model2_LSTM, self).__init__()
        self.name = "LSTM"
        self.backbone = backbone
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.lstm = nn.LSTM(input_size=backbone.num_features * 3, hidden_size=512, num_layers=1, batch_first=True, bidirectional=True)
        self.head = nn.Linear(1024, len(config.dataset.label_columns) * 3)
        
    def forward(self, **kwargs):
        batch = dict(**kwargs)
        features_output = []

        for series_description in ['Sagittal T2/STIR', 'Sagittal T1', 'Axial T2']:
            if series_description not in batch:
                features_output.append(None)
            else:
                images = batch[series_description]
                B, N_IMAGES, C, H, W = images.shape

                if config.model.train_backbone:
                    features = self.backbone.forward_features(images.view(B * N_IMAGES, C, H, W))
                else:
                    with torch.no_grad():
                        features = self.backbone.forward_features(images.view(B * N_IMAGES, C, H, W))

                features = self.pool(features).squeeze(-1).squeeze(-1).view(B, N_IMAGES, -1)

                # 如果该序列图像数量少于 15 张，进行填充
                if N_IMAGES < 15:
                    padding = torch.zeros((B, 15 - N_IMAGES, self.backbone.num_features), device=features.device)
                    features = torch.cat([features, padding], dim=1)

                features_output.append(features)

        # 将特征拼接在最后一个维度
        features_output = torch.cat(features_output, -1)

        lstm_out, _ = self.lstm(features_output)

        lstm_out = lstm_out.mean(1)
        output = self.head(lstm_out).view(-1, 10, 3)

        loss = None
        if 'labels' in batch:
            if config.model.use_weights:
                weight = torch.tensor([1., 2., 4.], device=output.device)
                loss_fn = torch.nn.CrossEntropyLoss(weight=weight)
            else:
                loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(output.view(-1, 3), batch['labels'].view(-1, ))

        return loss, output


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

# build model Vit

@todo

# crop

In [5]:
from torchvision import transforms

model_path = "/kaggle/input/resnet_for_crop/pytorch/default/1/resnet18_1_Input3Channels.pt"
import torch
import timm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import torchvision.transforms as transforms


class Croper:
    def __init__(self, model_path, visualize=False):
        self.device = torch.device("cpu")
        self.visualize = visualize  # 添加 visualize 参数
        # 创建 ResNet18 模型并加载到设备
        self.model = timm.create_model('resnet18', pretrained=True, num_classes=10)
        self.model.load_state_dict(torch.load(model_path, weights_only=True))
        self.model = self.model.to(self.device)   
        self.model.eval()

    def crop_saggital(self, images, h, w):
        results = []
        for dicom_data in images:
            image = dicom_data.pixel_array.astype(np.float32)  # H, W
            H, W = image.shape
            img_tensor = torch.tensor(image).unsqueeze(0)

            # 处理图像
            image_pre = (image - np.min(image)) / (np.max(image) - np.min(image))
            image_pre = torch.tensor(image_pre).unsqueeze(0)
            image_pre = F.interpolate(image_pre.unsqueeze(0), size=(256, 256), mode='bilinear', align_corners=False).squeeze(0)
            image_pre = image_pre.repeat(3, 1, 1).unsqueeze(0).to(self.device)

            with torch.no_grad():
                output = self.model(image_pre)
                output = torch.sigmoid(output)
            output_np = output.cpu().numpy().flatten()

            x = output_np[0::2] * W
            y = output_np[1::2] * H 
            points = np.stack([x, y], axis=1)

            # 执行裁剪
            for (px, py) in points:
                x_start = max(px - w // 2, 0)
                y_start = max(py - h // 2, 0)
                crop = torch.zeros((1, h, w), device=self.device)  # 将crop张量移动到与mean和std相同的设备
                crop_region = img_tensor[:, int(y_start):int(min(y_start + h, H)), int(x_start):int(min(x_start + w, W))]
                crop[:, :crop_region.shape[1], :crop_region.shape[2]] = crop_region.to(self.device)

                # Resize 裁剪后的图像到 (256, 256)
                crop = F.interpolate(crop.unsqueeze(0), size=(256, 256), mode='bilinear', align_corners=False).squeeze(0)
                # 扩展为 3 通道
                crop = crop.repeat(3, 1, 1)

                # Normalize
                mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1).to(self.device)
                std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1).to(self.device)
                crop = (crop - mean) / std

                results.append(crop)

            # 仅在 visualize 为 True 时可视化关键点和裁剪图像
            if self.visualize:
                self.viz_kpts(image, points, h, w)

        results = torch.stack(results)
        if self.visualize:
            self.viz_crop_images(results)

        return results


    def crop_axial(self, images, h, w):
        results = [] 
        for dicom_data in images:
            image = dicom_data.pixel_array.astype(np.float32)  # H, W
            H, W = image.shape
            img_tensor = torch.tensor(image, device=self.device).unsqueeze(0)  # 将 img_tensor 放在 self.device 上

            # 中心裁剪
            center_h, center_w = H // 2, W // 2
            start_h, end_h = max(0, center_h - h // 2), min(H, center_h + h // 2)
            start_w, end_w = max(0, center_w - w // 2), min(W, center_w + w // 2)
            cropped_img = img_tensor[:, start_h:end_h, start_w:end_w]

            # 调整裁剪后的图像尺寸
            if cropped_img.shape[1] != h or cropped_img.shape[2] != w:
                cropped_img = F.interpolate(cropped_img.unsqueeze(0), size=(h, w), mode='bilinear', align_corners=False).squeeze(0)

            # Resize 裁剪后的图像到 (256, 256)
            cropped_img = F.interpolate(cropped_img.unsqueeze(0), size=(256, 256), mode='bilinear', align_corners=False).squeeze(0)

            # 扩展为 3 通道
            cropped_img = cropped_img.repeat(3, 1, 1)

            # Normalize
            mean = torch.tensor([0.485, 0.456, 0.406], device=self.device).view(3, 1, 1)
            std = torch.tensor([0.229, 0.224, 0.225], device=self.device).view(3, 1, 1)
            cropped_img = (cropped_img - mean) / std

            results.append(cropped_img)

        # 如果需要可视化，则显示裁剪后的图像
        results = torch.stack(results, dim=0)
        if self.visualize:
            self.viz_crop_images(results)

        return results

    
    # 可视化关键点方法
    def viz_kpts(self, image, kpts, h, w):
        fig, ax = plt.subplots(1)
        ax.imshow(image, cmap="gray")
        for (x, y) in kpts:
            ax.plot(x, y, 'ro')
            rect = patches.Rectangle((x - w // 2, y - h // 2), w, h, linewidth=1, edgecolor='blue', facecolor='none')
            ax.add_patch(rect)
        plt.show()

    # 可视化裁剪图像方法
    def viz_crop_images(self, images):
        fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
        for i, img in enumerate(images):
            img_np = img.squeeze(0).cpu().numpy()
            axes[i].imshow(img_np, cmap="gray")
            axes[i].axis('off')
        plt.show()


        
cropper1 = Croper(model_path, visualize=False)

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

# data load
- preprocess: data augmentation(todo) / train test split
- data output

In [6]:
@dataclass
class ImagesDataset():
    study_ids: List[int]
    series_descriptions_df: pd.DataFrame
    path: Path
    is_train: bool
    transforms: A.Compose | None = None
    labels: pd.DataFrame | None = None
        
    def __post_init__(self):
        if self.transforms is None:
            self.transforms = lambda x: x
        self.h_s = 80
        self.w_s = 150
        self.h_a = 160
        self.w_a = 160
        
    def __len__(self):
        return len(self.study_ids)
    
    def __getitem__(self, index: int):
        study_id = self.study_ids[index]
        return_dict = {}

        for series_description in ['Sagittal T2/STIR', 'Sagittal T1', 'Axial T2']:
            rows = self.series_descriptions_df[
                (self.series_descriptions_df.study_id == study_id) & 
                (self.series_descriptions_df.series_description == series_description)
            ]

            if len(rows) == 0:
                # No images available for this description, create empty tensor with consistent shape
                if series_description in ['Sagittal T2/STIR', 'Sagittal T1']:
                    return_dict[series_description] = torch.zeros((10, 3, 256, 256))  # 统一尺寸为 (10, 3, 256, 256)
                elif series_description == 'Axial T2':
                    return_dict[series_description] = torch.zeros((10, 3, 256, 256))  # 统一尺寸为 (10, 3, 256, 256)

            else:
                # If images exist, process them
                row = rows.iloc[0]
                series_id = row.series_id
                use_path = self.path / ("train_images" if self.is_train else "test_images") / f"{study_id}/{series_id}"
                image_paths = sorted(glob.glob(str(use_path / "*.dcm")), key=lambda x: int(re.search(r"(\d+).dcm", x).group(1)))

                if series_description in ['Sagittal T2/STIR', 'Sagittal T1']:
                    center = len(image_paths) // 2
                    indices = [center - 1, center, center + 1]
                    image_paths = [image_paths[i] for i in indices]

                    # Read and crop images
                    images = [Path(image_path).dcmread() for image_path in image_paths]
                    cropped_images = cropper1.crop_saggital(images, self.h_s, self.w_s)  # Should return (N, 1, H, W)
                    # cropped_images = cropped_images.repeat(1, 3, 1, 1)  # 将单通道扩展为 3 通道
                    # return_dict[series_description] = cropped_images
                    if cropped_images.size(0) < 10:
                        padding = torch.zeros((10 - cropped_images.size(0), 3, 256, 256))
                        return_dict[series_description] = torch.cat([cropped_images, padding], dim=0)
                    else:
                        return_dict[series_description] = cropped_images[:10]
                    
                elif series_description == 'Axial T2':
                    num_images = len(image_paths)
                    if num_images > 10:
                        indices = [int(i * num_images / 10) for i in range(10)]
                        selected_paths = [image_paths[i] for i in indices]
                    else:
                        selected_paths = image_paths  # If fewer than 10 images, select all

                    images = [Path(image_path).dcmread() for image_path in selected_paths]
                    cropped_images = cropper1.crop_axial(images, self.h_a, self.w_a)
                    # cropped_images = cropped_images.repeat(1, 3, 1, 1)
                    if cropped_images.size(0) < 10:
                        padding = torch.zeros((10 - cropped_images.size(0), 3, 256, 256))
                        return_dict[series_description] = torch.cat([cropped_images, padding], dim=0)
                    else:
                        return_dict[series_description] = cropped_images[:10]

        # Attach labels if available
        if self.labels is not None:
            return_dict['labels'] = torch.tensor(self.labels.iloc[index].values, dtype=torch.long)

        return return_dict


def load_data(path: Path):
    sample_submission_df = pd.read_csv(path / 'sample_submission.csv')
    sample_submission_df = sample_submission_df[sample_submission_df['row_id'].str.contains('neural_foraminal_narrowing')].reset_index(drop=True)

    test_series_descriptions_df = pd.read_csv(path / 'test_series_descriptions.csv')
    train_df = pd.read_csv(path / 'train.csv')
    train_df = train_df[config.dataset.label_columns + ['study_id']].reset_index(drop=True)
    train_series_descriptions_df = pd.read_csv(path / 'train_series_descriptions.csv')
    
    return sample_submission_df, test_series_descriptions_df, train_df, train_series_descriptions_df


sample_submission_df, test_series_descriptions_df, train_df, train_series_descriptions_df = load_data(config.dataset.path)

if DEBUG:
    train_df = train_df.sample(100).reset_index(drop=True)

# train_df, valid_df
cv = GroupKFold(n_splits=config.n_folds)
train_df['fold'] = -1

for i, (train_idx, valid_idx) in enumerate(cv.split(train_df, groups=train_df['study_id'])):
    train_df.loc[valid_idx, 'fold'] = i
    

In [7]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches


dataset = ImagesDataset(
    study_ids=train_df.study_id.values.tolist(),
    path=config.dataset.path,
    is_train=True,
    series_descriptions_df=train_series_descriptions_df,
    transforms=transforms,
    labels=train_df[config.dataset.label_columns].fillna(-100).apply(lambda xx: [config.dataset.label_map[x] for x in xx], raw=True)
)

index = dataset.study_ids.index(1028684462)
data = dataset[index]

In [8]:
sample_submission_df.head(10)

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.333333,0.333333,0.333333
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.333333,0.333333,0.333333
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.333333,0.333333,0.333333
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.333333,0.333333,0.333333
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.333333,0.333333,0.333333
5,44036939_right_neural_foraminal_narrowing_l1_l2,0.333333,0.333333,0.333333
6,44036939_right_neural_foraminal_narrowing_l2_l3,0.333333,0.333333,0.333333
7,44036939_right_neural_foraminal_narrowing_l3_l4,0.333333,0.333333,0.333333
8,44036939_right_neural_foraminal_narrowing_l4_l5,0.333333,0.333333,0.333333
9,44036939_right_neural_foraminal_narrowing_l5_s1,0.333333,0.333333,0.333333


In [9]:
test_series_descriptions_df.head(10)

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [10]:
train_df.head(10)

,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,left_neural_foraminal_narrowing_l5_s1,right_neural_foraminal_narrowing_l1_l2,right_neural_foraminal_narrowing_l2_l3,right_neural_foraminal_narrowing_l3_l4,right_neural_foraminal_narrowing_l4_l5,right_neural_foraminal_narrowing_l5_s1,study_id,fold
0,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild,4003253,0
1,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild,4646740,2
2,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,7143189,1
3,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,8785691,0
4,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,10728036,4
5,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,11340341,3
6,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,11943292,2
7,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild,13317052,1
8,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,22191399,1
9,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,26342422,0


In [11]:
train_series_descriptions_df.head(10)

,study_id,series_id,series_description
0,4003253,702807833,Sagittal T2/STIR
1,4003253,1054713880,Sagittal T1
2,4003253,2448190387,Axial T2
3,4646740,3201256954,Axial T2
4,4646740,3486248476,Sagittal T1
5,4646740,3666319702,Sagittal T2/STIR
6,7143189,132939515,Sagittal T2/STIR
7,7143189,1951927562,Axial T2
8,7143189,3219733239,Sagittal T1
9,8785691,481125819,Sagittal T2/STIR


# Training & eval

In [12]:
import itertools

def build_gt_submission_df(df: pd.DataFrame) -> pd.DataFrame:
 
    df = df.drop('fold', axis=1).set_index('study_id').T
    rows = itertools.product(df.columns, df.index)
    values = df.values.flatten()

    result_df = pd.DataFrame({
        'row_id': [f'{study_id}_{location}' for study_id, location in rows],
        'normal_mild': [1 if value == 'Normal/Mild' else 0 for value in values],
        'moderate': [1 if value == 'Moderate' else 0 for value in values],
        'severe': [1 if value == 'Severe' else 0 for value in values],
    })
    
    result_df.loc[
        (result_df['normal_mild'] == 0)
        & (result_df['moderate'] == 0)
        & (result_df['severe'] == 0),
        'normal_mild'
    ] = 1
    
    return result_df
    
def prepare_dataset(df, series_descriptions_df, config, is_train=True):
    dataset = ImagesDataset(
        study_ids=df.study_id.values.tolist(),
        path=config.dataset.path,
        is_train=is_train,
        series_descriptions_df=series_descriptions_df,
        # use_n_images=config.dataset.use_n_images,
        transforms=transforms,
        labels=df[config.dataset.label_columns].fillna(-100).apply(lambda xx: [config.dataset.label_map[x] for x in xx], raw=True)
    )
    gt_submission = build_gt_submission_df(df)
    gt_submission['sample_weight'] = gt_submission.apply(lambda row: row['normal_mild'] + row['moderate'] * 2 + row['severe'] * 4, axis=1)
    return dataset, gt_submission

class ParticipantVisibleError(Exception):
    pass


def get_condition(full_location: str) -> str:
    # Given an input like spinal_canal_stenosis_l1_l2 extracts 'spinal'
    for injury_condition in ['spinal', 'foraminal', 'subarticular']:
        if injury_condition in full_location:
            return injury_condition
    raise ValueError(f'condition not found in {full_location}')


def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        row_id_column_name: str,
        any_severe_scalar: float
    ) -> float:
    '''
    Pseudocode:
    1. Calculate the sample weighted log loss for each medical condition:
    2. Derive a new any_severe label.
    3. Calculate the sample weighted log loss for the new any_severe label.
    4. Return the average of all of the label group log losses as the final score, normalized for the number of columns in each group.
       This mitigates the impact of spinal stenosis having only half as many columns as the other two conditions.
    '''

    target_levels = ['normal_mild', 'moderate', 'severe']

    # Run basic QC checks on the inputs
    if not pd.api.types.is_numeric_dtype(submission[target_levels].values):
        raise ParticipantVisibleError('All submission values must be numeric')

    if not np.isfinite(submission[target_levels].values).all():
        raise ParticipantVisibleError('All submission values must be finite')

    if solution[target_levels].min().min() < 0:
        raise ParticipantVisibleError('All labels must be at least zero')
    if submission[target_levels].min().min() < 0:
        raise ParticipantVisibleError('All predictions must be at least zero')

    solution['study_id'] = solution['row_id'].apply(lambda x: x.split('_')[0])
    solution['location'] = solution['row_id'].apply(lambda x: '_'.join(x.split('_')[1:]))
    solution['condition'] = solution['row_id'].apply(get_condition)

    del solution[row_id_column_name]
    del submission[row_id_column_name]
    assert sorted(submission.columns) == sorted(target_levels)

    submission['study_id'] = solution['study_id']
    submission['location'] = solution['location']
    submission['condition'] = solution['condition']

    condition_losses = []
    condition_weights = []
    for condition in ['foraminal']:
        condition_indices = solution.loc[solution['condition'] == condition].index.values
        condition_loss = sklearn.metrics.log_loss(
            y_true=solution.loc[condition_indices, target_levels].values,
            y_pred=submission.loc[condition_indices, target_levels].values,
            sample_weight=solution.loc[condition_indices, 'sample_weight'].values
        )
        condition_losses.append(condition_loss)
        condition_weights.append(1)

    return np.average(condition_losses, weights=condition_weights)

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, 2, keepdims=True))
    return e_x / e_x.sum(2, keepdims=True)

def compute_metrics(eval_pred, valid_gt_submission):
    logits, _ = eval_pred 
    # print(logits[0:10, :, :])
    logits = softmax(logits)
    # print(logits[0:10, :, :])

    # 将预测结果展开到表格中
    submission_df = pd.DataFrame({
        'row_id': valid_gt_submission['row_id'].values.tolist(),
        'normal_mild': logits[:, :, 0].flatten(),
        'moderate': logits[:, :, 1].flatten(),
        'severe': logits[:, :, 2].flatten(),
    })

    # print(valid_gt_submission.head(10))
    # print(submission_df.head(10))

    # 计算返回valid_loss
    return {
        'log_loss': score(valid_gt_submission.copy(), submission_df.copy(), 'row_id', 1.0)
    }

def train_fold(fold, train_df, train_series_descriptions_df, config, args):
    
    # 根据fold划分train和valid(方便进行 cross-validation,选择某个fold的数据作为valid)
    fold_train_df = train_df[train_df.fold != fold].reset_index(drop=True)
    fold_valid_df = train_df[train_df.fold == fold].reset_index(drop=True)

    fold_train_series_descriptions_df = train_series_descriptions_df[train_series_descriptions_df.study_id.isin(fold_train_df.study_id)]
    fold_valid_series_descriptions_df = train_series_descriptions_df[train_series_descriptions_df.study_id.isin(fold_valid_df.study_id)]

    train_dataset, _ = prepare_dataset(fold_train_df, fold_train_series_descriptions_df, config)
    valid_dataset, valid_gt_submission = prepare_dataset(fold_valid_df, fold_valid_series_descriptions_df, config)

    model = Model2_LSTM(backbone) # can change to diffrent model

    trainer = Trainer(
        model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        compute_metrics=lambda eval_pred: compute_metrics(eval_pred, valid_gt_submission),
    )

    trainer.train()
    
    # torch.save(model.state_dict(), f"best_model_{model.name}_fold_{fold}.pt")
    torch.save(model, f"best_model.pt")

In [13]:
for fold in range(1):
    train_fold(fold, train_df, train_series_descriptions_df, config, args)

Step,Training Loss,Validation Loss,Log Loss,Runtime,Samples Per Second,Steps Per Second
100,0.877400,0.808437,1.037323,303.220000,1.303000,0.043000
200,0.780500,0.794932,1.046200,242.380500,1.630000,0.054000
300,0.741500,0.794342,1.113880,245.929400,1.606000,0.053000
400,0.723400,0.787027,1.156527,245.047200,1.612000,0.053000


# ensemble